# Generatyviniai tinklai

Pasikartojantys neuroniniai tinklai (RNN) ir jų užtvarų ląstelių variantai, tokie kaip ilgos trumpos atminties ląstelės (LSTM) ir užtvarų pasikartojančios vienetai (GRU), suteikė mechanizmą kalbos modeliavimui, t. y. jie gali išmokti žodžių tvarką ir pateikti prognozes apie kitą žodį sekoje. Tai leidžia mums naudoti RNN **generatyvinėms užduotims**, tokioms kaip įprastas teksto generavimas, mašininis vertimas ir net vaizdų aprašymas.

RNN architektūroje, kurią aptarėme ankstesniame skyriuje, kiekvienas RNN vienetas generavo kitą paslėptą būseną kaip išvestį. Tačiau mes taip pat galime pridėti kitą išvestį prie kiekvieno pasikartojančio vieneto, kuris leistų mums generuoti **seką** (kuri yra tokio pat ilgio kaip pradinė seka). Be to, galime naudoti RNN vienetus, kurie kiekviename žingsnyje nepriima įvesties, o tiesiog naudoja pradinį būsenos vektorių ir tada generuoja išvesties seką.

Šiame užrašų knygelėje mes sutelksime dėmesį į paprastus generatyvinius modelius, kurie padeda mums generuoti tekstą. Paprastumo dėlei sukurkime **simbolių lygmens tinklą**, kuris generuoja tekstą raidė po raidės. Mokymo metu mums reikia paimti tam tikrą teksto korpusą ir padalyti jį į raidžių sekas.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Charakterių žodyno kūrimas

Norint sukurti generatyvinį tinklą, veikiantį simbolių lygiu, tekstą reikia suskaidyti į atskirus simbolius, o ne žodžius. Tai galima padaryti apibrėžiant kitokį žodyną:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Pažiūrėkime pavyzdį, kaip galime užkoduoti tekstą iš mūsų duomenų rinkinio:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Generatyvios RNN mokymas

RNN mokysime generuoti tekstą tokiu būdu. Kiekviename žingsnyje imsime simbolių seką, kurios ilgis yra `nchars`, ir paprašysime tinklo sugeneruoti kitą išvesties simbolį kiekvienam įvesties simboliui:

![Paveikslėlis, rodantis RNN generavimo pavyzdį su žodžiu 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Priklausomai nuo konkretaus scenarijaus, galime norėti įtraukti specialius simbolius, tokius kaip *sekos pabaiga* `<eos>`. Mūsų atveju, mes tiesiog norime išmokyti tinklą generuoti begalinį tekstą, todėl kiekvienos sekos dydį nustatysime kaip `nchars` simbolių. Taigi, kiekvienas mokymo pavyzdys susidarys iš `nchars` įvesties ir `nchars` išvesties (kurios yra įvesties seka, paslinkta vienu simboliu į kairę). Minipartija susidarys iš kelių tokių sekų.

Minipartijas generuosime taip: imsime kiekvieną naujienų tekstą, kurio ilgis yra `l`, ir iš jo sukursime visas galimas įvesties-išvesties kombinacijas (jų bus `l-nchars`). Jos sudarys vieną minipartiją, o minipartijų dydis kiekviename mokymo žingsnyje bus skirtingas.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Dabar apibrėžkime generatoriaus tinklą. Jis gali būti pagrįstas bet kuria pasikartojančia ląstele, kurią aptarėme ankstesniame skyriuje (paprasta, LSTM arba GRU). Mūsų pavyzdyje naudosime LSTM.

Kadangi tinklas kaip įvestį naudoja simbolius, o žodyno dydis yra gana mažas, mums nereikia įterpimo sluoksnio – vieno karšto kodavimo įvestis gali tiesiogiai pereiti į LSTM ląstelę. Tačiau, kadangi kaip įvestį perduodame simbolių numerius, prieš perduodant juos į LSTM, turime juos užkoduoti vieno karšto kodavimu. Tai atliekama kviečiant funkciją `one_hot` vykdymo metu (`forward` pass). Išvesties koduotojas bus linijinis sluoksnis, kuris paslėptą būseną pavers vieno karšto kodavimo išvestimi.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Mokymosi metu norime turėti galimybę generuoti tekstą. Tam apibrėšime funkciją `generate`, kuri sukurs išvesties eilutę, kurios ilgis yra `size`, pradedant nuo pradinės eilutės `start`.

Štai kaip tai veikia. Pirmiausia, visą pradinę eilutę perduosime per tinklą, gausime išvesties būseną `s` ir kitą numatomą simbolį `out`. Kadangi `out` yra vieno karšto kodavimo (one-hot encoded) formatu, naudojame `argmax`, kad gautume simbolio `nc` indeksą žodyne, o tada naudojame `itos`, kad nustatytume tikrąjį simbolį ir pridėtume jį prie rezultatų simbolių sąrašo `chars`. Šis simbolio generavimo procesas kartojamas `size` kartų, kad būtų sugeneruotas reikiamas simbolių skaičius.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Dabar pradėkime mokymą! Mokymo ciklas beveik toks pat kaip ir visuose ankstesniuose pavyzdžiuose, tačiau vietoj tikslumo kas 1000 epochų spausdiname sugeneruotą tekstą.

Ypatingą dėmesį reikia skirti tam, kaip apskaičiuojame nuostolį. Turime apskaičiuoti nuostolį, turėdami vieno karšto kodavimo išvestį `out` ir tikėtiną tekstą `text_out`, kuris yra simbolių indeksų sąrašas. Laimei, `cross_entropy` funkcija tikisi neapdorotos tinklo išvesties kaip pirmo argumento ir klasės numerio kaip antro, kas būtent ir atitinka mūsų situaciją. Ji taip pat automatiškai atlieka vidurkinimą pagal mini partijos dydį.

Taip pat apribojame mokymą iki `samples_to_train` pavyzdžių, kad nereikėtų per ilgai laukti. Skatiname jus eksperimentuoti ir bandyti ilgesnį mokymą, galbūt kelias epochas (tokiu atveju reikėtų sukurti dar vieną ciklą aplink šį kodą).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Šis pavyzdys jau generuoja gana gerą tekstą, tačiau jį galima dar labiau patobulinti keliais būdais:

* **Geresnė minibatch generacija**. Duomenų paruošimas mokymui buvo atliekamas generuojant vieną minibatch iš vieno pavyzdžio. Tai nėra idealu, nes minibatch dydžiai yra skirtingi, o kai kurie jų net negali būti sugeneruoti, nes tekstas yra trumpesnis nei `nchars`. Be to, maži minibatch nepakankamai apkrauna GPU. Protingiau būtų paimti didelį teksto fragmentą iš visų pavyzdžių, tada sugeneruoti visas įvesties-išvesties poras, jas sumaišyti ir sukurti vienodo dydžio minibatch.

* **Daugiasluoksnis LSTM**. Verta išbandyti 2 ar 3 LSTM ląstelių sluoksnius. Kaip minėjome ankstesniame skyriuje, kiekvienas LSTM sluoksnis iš tekstų išskiria tam tikrus modelius, o simbolių lygio generatoriaus atveju galima tikėtis, kad žemesnis LSTM lygis bus atsakingas už skiemenų išskyrimą, o aukštesni lygiai - už žodžius ir jų kombinacijas. Tai galima paprastai įgyvendinti perduodant sluoksnių skaičiaus parametrą LSTM konstruktoriui.

* Taip pat galite eksperimentuoti su **GRU vienetais** ir patikrinti, kurie veikia geriau, bei su **skirtingais paslėpto sluoksnio dydžiais**. Per didelis paslėpto sluoksnio dydis gali sukelti per didelį pritaikymą (pvz., tinklas išmoks tikslų tekstą), o per mažas dydis gali neduoti gero rezultato.


## Minkštas teksto generavimas ir temperatūra

Ankstesnėje `generate` funkcijos apibrėžtyje mes visada pasirinkdavome simbolį su didžiausia tikimybe kaip kitą simbolį generuojamame tekste. Tai dažnai lėmė, kad tekstas "kartodavosi" tarp tų pačių simbolių sekų vėl ir vėl, kaip šiame pavyzdyje:
```
today of the second the company and a second the company ...
```

Tačiau, jei pažvelgsime į tikimybių pasiskirstymą kitam simboliui, gali būti, kad skirtumas tarp kelių didžiausių tikimybių nėra didelis, pvz., vienas simbolis gali turėti tikimybę 0.2, o kitas - 0.19 ir pan. Pavyzdžiui, ieškant kito simbolio sekoje '*play*', kitas simbolis gali būti tiek tarpas, tiek **e** (kaip žodyje *player*).

Tai leidžia daryti išvadą, kad ne visada yra "teisinga" pasirinkti simbolį su didžiausia tikimybe, nes pasirinkus antrą pagal dydį tikimybę vis tiek galime gauti prasmingą tekstą. Protingiau yra **imti mėginius** iš tikimybių pasiskirstymo, kurį pateikia tinklo išvestis.

Šis mėginių ėmimas gali būti atliekamas naudojant `multinomial` funkciją, kuri įgyvendina vadinamąjį **multinominį pasiskirstymą**. Funkcija, kuri įgyvendina šį **minkštą** teksto generavimą, apibrėžta žemiau:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Mes pristatėme dar vieną parametrą, vadinamą **temperatūra**, kuris naudojamas nurodyti, kaip stipriai turėtume laikytis didžiausios tikimybės. Jei temperatūra yra 1.0, atliekame sąžiningą multinominį mėginių ėmimą, o kai temperatūra pasiekia begalybę - visos tikimybės tampa lygios, ir mes atsitiktinai pasirenkame kitą simbolį. Žemiau pateiktame pavyzdyje galime pastebėti, kad tekstas tampa beprasmiškas, kai temperatūra per daug padidėja, ir primena „ciklinį“ sunkiai generuojamą tekstą, kai ji artėja prie 0.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
